In [1]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout
from keras.utils import to_categorical

import numpy as np
from numpy import array

Using TensorFlow backend.


In [2]:
import re

In [3]:
corp = open('history.conll', encoding='utf-8').read()

from conllu import parse, parse_tree
from conllu import print_tree

tree = parse(corp)

import re
p = re.compile('[а-яА-ЯёЁ]')
p1=re.compile('[0-9]')

lemmas=[]
for i in tree:
    for j in i:
        if p.search(j['lemma']) and not p1.search(j['lemma']):
            lemmas.append(j['lemma'].lower())

In [4]:
from numpy import array

def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
    return in_text

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(i for i in lemmas)])
encoded = tokenizer.texts_to_sequences([' '.join(i for i in lemmas)])[0]
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
sequences = list()
for i in range(2, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)


Vocabulary Size: 8134
Total Sequences: 64202
Max Sequence Length: 2


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=75, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1, 10)             81340     
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_4 (Dense)              (None, 8134)              414834    
Total params: 508,374
Trainable params: 508,374
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/75
139s - loss: 7.5690 - acc: 0.0470
Epoch 2/75
143s - loss: 7.1808 - acc: 0.0494
Epoch 3/75
155s - loss: 7.0665 - acc: 0.0579
Epoch 4/75
148s - loss: 6.9562 - acc: 0.0626
Epoch 5/75
141s - loss: 6.8437 - acc: 0.0714
Epoch 6/75
144s - loss: 6.7199 - acc: 0.0785
Epoch 7/75
136s - loss: 6.5555 - acc: 0.0925
Epoch 8/75
138s - loss: 6.3808 - acc: 0.1072
Epoch 9/75
141s - loss: 6.2215 - ac

In [ ]:
model.save('lemmas_model2.h5')
#model.save('lemmas_model.h5')

In [6]:
from keras.models import load_model
model2=load_model('lemmas_model2.h5')

In [7]:
#так генерируется наиболее вероятное следующеее слово
print(generate_seq(model2, tokenizer, max_length-1, 'взгляд', 1))

взгляд на


In [8]:
#получаем матрицу вероятностей появления для всех известных модели слов после заданного н-грамма
def get_all_probs(model, tokenizer, max_length, seed_text):
    in_text = seed_text
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    yhat = model.predict(encoded, verbose=0)
    return yhat

In [9]:
np.sort(get_all_probs(model2, tokenizer, max_length-1, 'взгляд')[0])[-5:]

array([ 0.02563183,  0.04811867,  0.05004111,  0.1138985 ,  0.32530677], dtype=float32)

In [18]:
#генерируем строки на замену
import nltk
stop_words= nltk.corpus.stopwords.words('russian')
inv_dict = {v: k for k, v in tokenizer.word_index.items()}
from pymystem3 import Mystem
m = Mystem()

small=1e-10
#give it lemmatized strings maybe
def get_bigram(bigram_string):
    suggestions=[]
    #for string in trigram_strings:
    prob=0.0
    unigram=m.lemmatize(bigram_string[0])[0]
    last_word=m.lemmatize(bigram_string[-1])[0]
    prob_matrix=get_all_probs(model2, tokenizer, max_length-1, unigram)[0]
    most_probable=np.sort(prob_matrix)[-200:]
    if last_word in tokenizer.word_index.keys():
        prob=prob_matrix[tokenizer.word_index[last_word]]
    else:
        prob=small
    if prob not in most_probable and unigram not in stop_words:
        for i in most_probable[-3:]:
            ind=np.where(prob_matrix==i)[0][0]
            next_word = inv_dict[ind]
            if next_word not in stop_words:
                suggestions.append(unigram +' '+next_word)
    if suggestions:
        return 'возможно неверная коллокация: {}, варианты замен: {}'.format(bigram_string, suggestions)

In [11]:
get_bigram(['взгляд', 'православный'])

"возможно неверная коллокация: ['взгляд', 'православный'], варианты замен: ['взгляд николай']"

In [12]:
# import json
# with open('dict1.json','w',encoding='utf-8') as out:
#     json.dump(tokenizer.word_index,out)

In [13]:
#тестируем
test=open('бэд.txt', encoding='utf-8').read()

In [14]:
clean_text=[]
for i in test.split():
    i=i.lower()
    if p.search(i) and not p1.search(i):
        clean_text.append(i)

In [15]:
bigrams=list(nltk.ngrams(clean_text,2))

In [16]:
get_bigram(list(bigrams[10]))

"возможно неверная коллокация: ['всероссийский', 'умер'], варианты замен: ['всероссийский математика', 'всероссийский принц', 'всероссийский иерархичность', 'всероссийский вдова', 'всероссийский чрезвычайный']"

In [17]:
#np.sort(prob_matrix)[-100:]
#most_probable[-5:]
for i in range(30):
    print(get_bigram(list(bigrams[i])))

None
возможно неверная коллокация: ['особое', 'значение'], варианты замен: ['особый положение', 'особый назначение', 'особый место', 'особый внимание', 'особый военный']
возможно неверная коллокация: ['значение', 'приобретает'], варианты замен: ['значение второй']
возможно неверная коллокация: ['приобретает', 'вопрос'], варианты замен: ['приобретать нормативный', 'приобретать особый']
None
возможно неверная коллокация: ['о', 'заговоре'], варианты замен: ['о бодуэн', 'о деятельность', 'о работа']
возможно неверная коллокация: ['заговоре', 'против'], варианты замен: ['заговор ориентироваться', 'заговор разыскание', 'заговор сибирский', 'заговор шечкина', 'заговор бесовский']
возможно неверная коллокация: ['против', 'павла'], варианты замен: ['против русский', 'против партизан', 'против ссср', 'против японец', 'против большевизм']
возможно неверная коллокация: ['павла', 'император'], варианты замен: ['павел иванович', 'павел александрович', 'павел павлович', 'павел францевич', 'павел федо

In [19]:
for i in range(30):
    print(get_bigram(list(bigrams[i])))

None
None
None
возможно неверная коллокация: ['приобретает', 'вопрос'], варианты замен: ['приобретать нормативный', 'приобретать особый']
None
None
возможно неверная коллокация: ['заговоре', 'против'], варианты замен: ['заговор сибирский', 'заговор шечкина', 'заговор бесовский']
возможно неверная коллокация: ['против', 'павла'], варианты замен: ['против ссср', 'против японец', 'против большевизм']
возможно неверная коллокация: ['павла', 'император'], варианты замен: ['павел павлович', 'павел францевич', 'павел федорович']
возможно неверная коллокация: ['император', 'всероссийский'], варианты замен: ['император тойерданк', 'император ромеев', 'император максимилиан']
возможно неверная коллокация: ['всероссийский', 'умер'], варианты замен: ['всероссийский иерархичность', 'всероссийский вдова', 'всероссийский чрезвычайный']
None
None
возможно неверная коллокация: ['раскрытой', 'смертью,'], варианты замен: ['раскрывать резкий', 'раскрывать количественный', 'раскрывать деталь']
None
None
No